In [ ]:
import offscreen_renderer as ogl
from offscreen_renderer import GLenum as GLenum
import pickle, gzip
import numpy as np
from time import time

In [ ]:
width, height, P, N, C, modParams, camParams = pickle.load(gzip.open('data/render_data.pkl.gz'))

In [ ]:
mrender = ogl.MeshRenderer(2 * width, 2 * height)

In [ ]:
mrender.setMesh(P, None, N, C)

In [ ]:
mrender.lookAt(*camParams)
mrender.modelMatrix(*modParams)
mrender.perspective(50, width / height, 0.1, 2000)

In [ ]:
mrender.transparentBackground  = True

In [ ]:
mrender.alpha = 0.33
mrender.lineWidth = 1.0
mrender.render(True)

In [ ]:
mrender.orbitedLookAt(*camParams, np.pi / 16)
mrender.render(True)

In [ ]:
mrender.image()

In [ ]:
mrender.save('out.png')